In [1]:
from langchain_openai.chat_models import ChatOpenAI

import gpt4science.settings as settings

llm4 = ChatOpenAI(api_key=settings.OPENAI_API_KEY, model=settings.GPT4_TURBO)
llm3 = ChatOpenAI(api_key=settings.OPENAI_API_KEY, model=settings.GPT3_TURBO)

In [3]:
from pydantic import BaseModel, HttpUrl
import requests
from bs4 import BeautifulSoup
from typing import List, Optional

class ScholarResult(BaseModel):
    title: str
    url: Optional[HttpUrl]  # Use HttpUrl for URL validation

def get_scholar_results(query: str) -> List[ScholarResult]:
    url = 'https://scholar.google.com/scholar'
    headers = {'User-Agent': 'Mozilla/5.0'}
    params = {'q': query}
    response = requests.get(url, headers=headers, params=params)

    results = []
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        for item in soup.find_all('h3', class_='gs_rt'):
            title = item.get_text()
            link = item.a['href'] if item.a else None
            results.append(ScholarResult(title=title, url=link))
    return results

# Example usage
query = '"Quantum Gates" AND circuits'
scholar_results = get_scholar_results(query)
for result in scholar_results:
    print(f'Title: {result.title}\nURL: {result.url}\n')


Title: Quantum gates and circuits
URL: https://royalsocietypublishing.org/doi/abs/10.1098/rspa.1998.0159

Title: Constructing quantum circuits with global gates
URL: https://iopscience.iop.org/article/10.1088/1367-2630/abf1b3/meta

Title: Ultrafast quantum gates in circuit QED
URL: https://journals.aps.org/prl/abstract/10.1103/PhysRevLett.108.120501

Title: Quantum circuits with uniformly controlled one-qubit gates
URL: https://journals.aps.org/pra/abstract/10.1103/PhysRevA.71.052330

Title: Reducing the number of non-Clifford gates in quantum circuits
URL: https://journals.aps.org/pra/abstract/10.1103/PhysRevA.102.022406

Title: Learning Quantum Circuits of Some T Gates
URL: https://ieeexplore.ieee.org/abstract/document/9714418/

Title: Quantum gates
URL: https://link.springer.com/chapter/10.1007/978-1-84628-887-6_2

Title: Optimal quantum circuits for general two-qubit gates
URL: https://journals.aps.org/pra/abstract/10.1103/PhysRevA.69.032315

Title: Decompositions of general quantu

In [4]:
import requests

from bs4 import BeautifulSoup

res = requests.get("https://journals.aps.org/pra/abstract/10.1103/PhysRevA.69.032315")
soup = BeautifulSoup(res.text, 'html.parser')

# Extract the text content
text_content = soup.get_text()

# Print the cleaned text content
parsed_text = text_content.replace("\n", " ").replace("\t", " ").replace("\r", " ")
parsed_text

'  Phys. Rev. A 69, 032315 (2004)  -  Optimal quantum circuits for general two-qubit gates             Skip to Main ContentJournalsPhysical Review LettersPhysical Review XPRX EnergyPRX LifePRX QuantumReviews of Modern PhysicsPhysical Review APhysical Review BPhysical Review CPhysical Review DPhysical Review EPhysical Review ResearchPhysical Review Accelerators and BeamsPhysical Review AppliedPhysical Review FluidsPhysical Review MaterialsPhysical Review Physics Education ResearchPhysical ReviewPhysical Review (Series I)Physics Physique FizikaPhysics MagazineHelp/FeedbackSearch/Article LookupLog inPhysical Review Acovering atomic, molecular, and optical physics and quantum informationHighlightsLettersRecentAcceptedCollectionsAuthorsRefereesSearchPressAboutEditorial TeamOptimal quantum circuits for general two-qubit gatesFarrokh Vatan and Colin WilliamsPhys. Rev. A 69, 032315 – Published 22 March 2004  More×ArticleReferencesCiting Articles (236)ArticleReferencesCiting Articles (236)PDFHT

In [12]:
from pydantic.v1 import BaseModel
from datetime import date
from langchain.chains import create_extraction_chain_pydantic
from pprint import pprint

class PaperSource(BaseModel):
    title: str
    abstract: str
    authors: list[str]
    published_in: str
    published_date_year: int
    published_date_month: int
    published_date_day: int

    @property
    def published_date(self):
        return date(self.published_date_year, self.published_date_month, self.published_date_day)

# Example usage
chain = create_extraction_chain_pydantic(pydantic_schema=PaperSource, llm=llm3)
paper_source = chain.invoke(parsed_text)["text"][0]

In [14]:
pprint(paper_source.dict())

{'abstract': 'In order to demonstrate nontrivial quantum computations '
             'experimentally, such as the synthesis of arbitrary entangled '
             'states, it will be useful to understand how to decompose a '
             'desired quantum computation into the shortest possible sequence '
             'of one-qubit and two-qubit gates. We contribute to this effort '
             'by providing a method to construct an optimal quantum circuit '
             'for a general two-qubit gate that requires at most 3 '
             'controlled-NOT (CNOT) gates and 15 elementary one-qubit gates. '
             'Moreover, if the desired two-qubit gate corresponds to a purely '
             'real unitary transformation, we provide a construction that '
             'requires at most 2 CNOT and 12 one-qubit gates. We then prove '
             'that these constructions are optimal with respect to the family '
             'of CNOT, y-rotation, z-rotation, and phase gates.',
 'authors':